In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('output.csv')

In [3]:
df

,Unnamed: 0,Name,Suit,Points
0,0,Queen of Hearts,Hearts,0.0
1,1,9 of Diamonds,Diamonds,2.0
2,2,10 of Spades,Spades,1.0
3,3,Queen of Clubs,Clubs,0.0
4,4,7 of Clubs,Clubs,0.0
5,5,Ace of Hearts,Hearts,1.1
6,6,10 of Diamonds,Diamonds,1.0
7,7,9 of Clubs,Clubs,2.0
8,8,10 of Clubs,Clubs,1.0
9,9,8 of Hearts,Hearts,0.0


In [4]:
df = df.drop('Unnamed: 0', axis = 1)

In [5]:
class Player():
    def __init__(self, cards, name):
        self.cards = cards
        self.colour_made = False
        self.points_made = 0
        self.name = name
        self.hands_won = []
        self.last_hand_won = False

class Robot():
    def __init__(self, cards, name):
        self.cards = cards
        self.colour_made = False
        self.points_made = 0
        self.name = name
        self.hands_won = []
        self.last_hand_won = False
class Game():
    def __init__(self):
        self.color = None
        self.last_hand_winner = None
        self.digged = False


In [6]:
class Bid():
    def __init__(self, b1, b2):
        self.how_much_b1 = 17
        self.how_much_b2 = 17
        self.b1_bid = input(f"Do you want to bid {b1.name}? Y/N: ").lower()
        
        if self.b1_bid == "n":
            print(f"{b2.name} wins the bid as {b1.name} passes at {self.how_much_b2}")
            self.winner = b2
        elif self.b1_bid == "y":
            self.b2_bid = input(f"Do you want to bid {b2.name}? Y/N: ").lower()

            if self.b2_bid == "n":
                print(f"{b1.name} wins the bid as {b2.name} passes at {self.how_much_b1}")
                self.winner = b1
            else:
                while self.b1_bid == "y" and self.b2_bid == "y":
                    self.how_much_b1 = int(input(f"How much would you like to bid {b1.name}? (minimum bid is 17): "))
                    self.how_much_b2 = int(input(f"How much would you like to bid {b2.name}? (minimum bid is 17): "))

                    # Ensure minimum bid
                    while self.how_much_b1 < 17 or self.how_much_b2 < 17:
                        print("Invalid bid! The minimum bid is 17.")
                        self.how_much_b1 = int(input(f"How much would you like to bid {b1.name}? (minimum bid is 17): "))
                        self.how_much_b2 = int(input(f"How much would you like to bid {b2.name}? (minimum bid is 17): "))

                    # Determine winner of the bid
                    if self.how_much_b1 < self.how_much_b2:
                        self.b1_bid = input(f"{b2.name} is winning the bid. Do you want to continue, {b1.name}? Y/N: ").lower()
                        if self.b1_bid == "n":
                            print(f"{b2.name} wins the bid as {b1.name} passes at {self.how_much_b2}")
                            self.winner = b2
                    elif self.how_much_b1 > self.how_much_b2:
                        self.b2_bid = input(f"{b1.name} is winning the bid. Do you want to continue, {b2.name}? Y/N: ").lower()
                        if self.b2_bid == "n":
                            print(f"{b1.name} wins the bid as {b2.name} passes at {self.how_much_b1}")
                            self.winner = b1


In [7]:
game = Game()

In [8]:
def shuffle_deck():
    global df
    df = df.sample(frac = 1, ignore_index = True)

In [9]:
def distribute_cards():
    global df
    global player, robot_1, robot_2, robot_3
    player = Player(df[0:8], input("What is your name? "))
    robot_1 = Robot(df[8:16], 'Robot 1')
    robot_2 = Robot(df[16:24], 'Robot 2')
    robot_3 = Robot(df[24:32], 'Robot 3')

In [10]:
def play_card(name, player):
    global current_hand
    current_hand = pd.concat([current_hand, df[df['Name'] == name]], ignore_index = True)
    # pd.concat([new_data, data], ignore_index=True)
    player.cards = player.cards.drop(player.cards[player.cards['Name'] == name].index)

In [11]:
def setup():
    shuffle_deck() # Shuffling all the cards
    distribute_cards() # Distributing all the cards
    print("Nice to meet you {}".format(player.name))
    print("Welcome to our newly made game of 29. We hope you enjoy it")
    print(player.cards[:4]) # Showing the player their first 4 cards
    global game
    
    # BIDDING
    player_bid = int(input('How much do you want to bid?: '))
    
    # robot_1
    r1_bid = 0
    if len(robot_1.cards[:4]['Suit'].mode() == 1):
        r1_mode_suit = robot_1.cards[:4]['Suit'].mode().iloc[0]
        r1_color = robot_1.cards[:4]['Suit'].value_counts()[r1_mode_suit]
        r1_points = robot_1.cards[:4][robot_1.cards[:4]['Suit'] == r1_mode_suit]['Points'].sum()
        
        if r1_points == 3 or r1_points == 3.1:
            if r1_color == 2: r1_bid = 18
            if r1_color == 3: r1_bid = 19
        if r1_points == 4 or r1_points == 4.1:
            if r1_color == 2: r1_bid = 18
            if r1_color == 3: r1_bid = 19
        if r1_points == 5 or r1_points == 5.1:
            if r1_color == 2: r1_bid = 19
            if r1_color == 3: r1_bid = 20
        if r1_points == 6 or r1_points == 6.1:
            if r1_color == 3: r1_bid = 21
        if r1_points == 6 or r1_points == 6.1 or r1_points == 7.1:
            if r1_color == 4: r1_bid = 22
    
    # robot_2
    r2_bid = 0
    if len(robot_2.cards[:4]['Suit'].mode() == 2):
        r2_mode_suit = robot_2.cards[:4]['Suit'].mode().iloc[0]
        r2_color = robot_2.cards[:4]['Suit'].value_counts()[r2_mode_suit]
        r2_points = robot_2.cards[:4][robot_2.cards[:4]['Suit'] == r2_mode_suit]['Points'].sum()
        
        if r2_points == 3 or r2_points == 3.1:
            if r2_color == 2: r2_bid = 18
            if r2_color == 3: r2_bid = 19
        if r2_points == 4 or r2_points == 4.1:
            if r2_color == 2: r2_bid = 18
            if r2_color == 3: r2_bid = 19
        if r2_points == 5 or r2_points == 5.1:
            if r2_color == 2: r2_bid = 19
            if r2_color == 3: r2_bid = 20
        if r2_points == 6 or r2_points == 6.1:
            if r2_color == 3: r2_bid = 21
        if r2_points == 6 or r2_points == 6.1 or r2_points == 7.1:
            if r2_color == 4: r2_bid = 22
    
    # robot_3
    r3_bid = 0
    if len(robot_3.cards[:4]['Suit'].mode() == 3):
        r3_mode_suit = robot_3.cards[:4]['Suit'].mode().iloc[0]
        r3_color = robot_3.cards[:4]['Suit'].value_counts()[r3_mode_suit]
        r3_points = robot_3.cards[:4][robot_3.cards[:4]['Suit'] == r3_mode_suit]['Points'].sum()
        
        if r3_points == 3 or r3_points == 3.1:
            if r3_color == 2: r3_bid = 18
            if r3_color == 3: r3_bid = 19
        if r3_points == 4 or r3_points == 4.1:
            if r3_color == 2: r3_bid = 18
            if r3_color == 3: r3_bid = 19
        if r3_points == 5 or r3_points == 5.1:
            if r3_color == 2: r3_bid = 19
            if r3_color == 3: r3_bid = 20
        if r3_points == 6 or r3_points == 6.1:
            if r3_color == 3: r3_bid = 22
        if r3_points == 6 or r3_points == 6.1 or r3_points == 7.1:
            if r3_color == 4: r3_bid = 22
    bidding = [player_bid, r1_bid, r2_bid, r3_bid]
    max_bid = max(bidding)
    max_bid_index = bidding.index(max_bid)
    if max_bid_index == 0: winner = player.name
    if max_bid_index == 1: winner = robot_1.name
    if max_bid_index == 2: winner = robot_2.name
    if max_bid_index == 3: winner = robot_3.name
    print('{} won the bet and will make the color on {} points.'.format(winner, max_bid))
    if winner == player.name:
        player.colour_made = True
        game.color = input(f"What is the colour {player.name}?")
    if winner == robot_1.name:
        game.color = r1_mode_suit
    if winner == robot_2.name:
        game.color = r2_mode_suit
    if winner == robot_3.name:
        game.color = r3_mode_suit
    game.color = game.color.capitalize()

In [16]:
def flow_chart(r):
    global current_hand, game
    global df
    r.suits = list(r.cards["Suit"])
    if len(current_hand) ==0:
        current_suit = "Hearts"
    else:
        
        current_suit = current_hand.iloc[0]['Suit']
    
    if current_suit in r.suits:
        how_many = r.suits.count(current_suit)
        
        if how_many == 1:
            card_to_play = r.cards[r.cards['Suit'] == current_suit]['Name']
            card_to_play = card_to_play.iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play, r)

        if how_many > 1:
            
            if len(r.cards[(r.cards['Suit'] == current_suit) & (r.cards['Points'] == 3)]) == 0:
                cs = r.cards[r.cards['Suit'] == current_suit]
                card_to_play = cs[cs['Points'] == cs['Points'].min()].sample(n = 1)['Name']
                card_to_play = card_to_play.iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)
            
            else: 
                card_to_play = r.cards[(r.cards['Suit'] == current_suit) & (r.cards['Points'] == 3)]['Name']
                card_to_play = card_to_play.iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)
            
    if not game.digged and current_suit not in r.suits:
        print(f'{r.name} has revealed the trump card. It is {game.color}')
        if len(r.cards[(r.cards['Suit'] == game.color)]) > 0:
            cs = r.cards[r.cards['Suit'] == game.color]
            card_to_play = cs[cs['Points'] == cs['Points'].min()].sample(n = 1)['Name']
            card_to_play = card_to_play.iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play, r)
        else: 
            card_to_play = r.cards.sample(n = 1)['Name'].iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play, r)
        game.digged = True
    elif game.digged and current_suit not in r.suits:
        if len(r.cards[(r.cards['Suit'] == game.color)]) > 0:
            cs = r.cards[r.cards['Suit'] == game.color]
            card_to_play = cs[cs['Points'] == cs['Points'].min()].sample(n = 1)['Name']
            card_to_play = card_to_play.iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play, r)
        else: 
            card_to_play = r.cards.sample(n = 1)['Name'].iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play, r)

In [13]:
current_hand = pd.DataFrame(data = {}, columns = df.columns)

In [14]:
def play_game():
    import warnings

    warnings.filterwarnings(
        "ignore",
        category=FutureWarning, 
        message="The behavior of DataFrame concatenation with empty or all-NA entries is deprecated"
    )

    #SETUP THE GAME
    setup()
    #DO THE GLOBALS
    global player, robot_1, robot_2, robot_3, game, current_hand, last_hand
    #print(player.cards)
    #PLAYE ALL THE 8 HANDS
    for i in range(0, 8):
        print(f"Here are your cards {player.name}: \n {player.cards}")
       
        if player.last_hand_won: 
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
            play_card(card, player)
            flow_chart(robot_1)
            flow_chart(robot_2)
            flow_chart(robot_3)
            index_selctor = {"player":0,"robot_1":1,"robot_2":1,"robot_3":3}
        if robot_1.last_hand_won: 
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
            flow_chart(robot_1)
            flow_chart(robot_2)
            flow_chart(robot_3)
            play_card(card, player)
            index_selctor = {"robot_1":0,"robot_2":1,"robot_3":2,"player":3}
        if robot_2.last_hand_won:
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
            flow_chart(robot_2)
            flow_chart(robot_3)
            play_card(card, player)
            flow_chart(robot_1)
            index_selctor = {"robot_1":0,"robot_2":1,"player":2,"robot_3":3}
        if robot_3.last_hand_won:
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
            flow_chart(robot_3)
            play_card(card, player)
            flow_chart(robot_1)
            flow_chart(robot_2)
            index_selctor = {"robot_1":0,"player":1,"robot_2":2,"robot_3":3}
        else: 
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
            play_card(card, player)
            flow_chart(robot_1)
            flow_chart(robot_2)
            flow_chart(robot_3)
            index_selctor = {"player":0,"robot_1":1,"robot_2":1,"robot_3":3}
        print(current_hand)
        if game.digged and game.color in current_hand:
            find_in = current_hand[current_hand['Suit'] == game.color]
            winner_index = find_in[find_in['Points'] == find_in['Points'].max()].index
        elif not game.color in current_hand:
            c_s = current_hand['Suit'].iloc[0]
            find_in = current_hand[current_hand['Suit'] == c_s]
            winner_index = find_in[find_in['Points'] == find_in['Points'].max()].index
        if winner_index == index_selctor["player"]: 
                for c in list(current_hand["Name"]): 
                    player.hands_won.append(c)
                    player.last_hand_won = True
                    robot_1.last_hand_won = False
                    robot_2.last_hand_won = False
                    robot_3.last_hand_won = False
        if winner_index == index_selctor["robot_1"]: 
                for c in  list(current_hand["Name"]):
                    robot_1.hands_won.append(c)
                    player.last_hand_won = False
                    robot_1.last_hand_won = True
                    robot_2.last_hand_won = False
                    robot_3.last_hand_won = False
        if winner_index == index_selctor["robot_2"]: 
                for c in list(current_hand["Name"]):
                    robot_2.hands_won.append(c)
                    player.last_hand_won = False
                    robot_1.last_hand_won = False
                    robot_2.last_hand_won = True
                    robot_3.last_hand_won = False
        if winner_index == index_selctor["robot_3"]: 
                for c in list(current_hand["Name"]): 
                    robot_3.hands_won.append(c)
                    player.last_hand_won = False
                    robot_1.last_hand_won = False
                    robot_2.last_hand_won = False
                    robot_3.last_hand_won = True
        
            
            
        last_hand = current_hand
        current_hand = pd.DataFrame(data = {}, columns = df.columns)

In [15]:
play_game()

What is your name? o
Nice to meet you o
Welcome to our newly made game of 29. We hope you enjoy it
              Name      Suit  Points
0  Queen of Spades    Spades     0.0
1    9 of Diamonds  Diamonds     2.0
2      9 of Hearts    Hearts     2.0
3  Queen of Hearts    Hearts     0.0
How much do you want to bid?: 22
o won the bet and will make the color on 22 points.
What is the colour o?hearts
Here are your cards o: 
               Name      Suit  Points
0  Queen of Spades    Spades     0.0
1    9 of Diamonds  Diamonds     2.0
2      9 of Hearts    Hearts     2.0
3  Queen of Hearts    Hearts     0.0
4  Ace of Diamonds  Diamonds     1.1
5   Queen of Clubs     Clubs     0.0
6      9 of Spades    Spades     2.0
7   10 of Diamonds  Diamonds     1.0
What card do you wanna play o9 of Hearts
Robot 1 plays 8 of Hearts
Robot 2 plays Ace of Hearts
Robot 3 plays Jack of Hearts
             Name    Suit  Points
0     9 of Hearts  Hearts     2.0
1     8 of Hearts  Hearts     0.0
2   Ace of Hearts  

IndexError: single positional indexer is out-of-bounds

In [29]:
winner_index = df[(df['Suit'] == game.color) & (df['Points'] == df['Points'].max())].index

In [30]:
winner_index

Index([23], dtype='int64')

In [31]:
df[(df['Suit'] == game.color) & (df['Points'] == df['Points'].max())]

,Name,Suit,Points
23,Jack of Spades,Spades,3.0


In [38]:
index_selctor = {"robot_1":0,"player":1,"robot_2":2,"robot_3":23}

In [ ]:
current_hand[(current_hand['Suit'] == current_hand.iloc[0]['Suit']) & (current_hand['Points'] == current_hand['Points'].max())]['Points'].index

In [46]:
# Ensure Points is numeric and handle missing values if needed
last_hand['Points'] = pd.to_numeric(last_hand['Points'], errors='coerce')

# Get the max points for the current suit
max_points = last_hand[last_hand['Suit'] == last_hand.iloc[0]['Suit']]['Points'].max()

# Find the index of the card with max points in the same suit
winner_index = last_hand[(last_hand['Suit'] == last_hand.iloc[0]['Suit']) & (last_hand['Points'] == max_points)].index


NameError: name 'last_hand' is not defined

In [49]:
game.digged = True

In [50]:
current_hand

,Name,Suit,Points


In [51]:
# Ensure Points is numeric and handle missing values if needed
current_hand['Points'] = pd.to_numeric(current_hand['Points'], errors='coerce')

# Get the max points for the current suit
max_points = current_hand[current_hand['Suit'] == current_hand.iloc[0]['Suit']]['Points'].max()

# Find the index of the card with max points in the same suit
winner_index = current_hand[(current_hand['Suit'] == current_hand.iloc[0]['Suit']) & (current_hand['Points'] == max_points)].index



IndexError: single positional indexer is out-of-bounds

In [56]:
import pandas as pd
import random

# Define the data
cards = [
    "Ace of Spades", "2 of Spades", "3 of Spades", "4 of Spades", "5 of Spades", 
    "6 of Spades", "7 of Spades", "8 of Spades", "9 of Spades", "10 of Spades", 
    "Jack of Spades", "Queen of Spades", "King of Spades",
    "Ace of Hearts", "2 of Hearts", "3 of Hearts", "4 of Hearts", "5 of Hearts", 
    "6 of Hearts", "7 of Hearts", "8 of Hearts", "9 of Hearts", "10 of Hearts", 
    "Jack of Hearts", "Queen of Hearts", "King of Hearts",
    "Ace of Diamonds", "2 of Diamonds", "3 of Diamonds", "4 of Diamonds", "5 of Diamonds", 
    "6 of Diamonds", "7 of Diamonds", "8 of Diamonds", "9 of Diamonds", "10 of Diamonds", 
    "Jack of Diamonds", "Queen of Diamonds", "King of Diamonds",
    "Ace of Clubs", "2 of Clubs", "3 of Clubs", "4 of Clubs", "5 of Clubs", 
    "6 of Clubs", "7 of Clubs", "8 of Clubs", "9 of Clubs", "10 of Clubs", 
    "Jack of Clubs", "Queen of Clubs", "King of Clubs"
]

# Create a random selection of 5 cards (or adjust the number of rows as needed)
random_cards = random.sample(cards, 5)

# Define the specific points to randomly assign
points_options = [0, 1, 1.1, 3]

# Create the DataFrame
test_hand = pd.DataFrame({
    "Name": random_cards,
    "Suit": [card.split()[-1] for card in random_cards],
    "Points": [random.choice(points_options) for _ in random_cards]  # Random points from the specified options
})

print(test_hand)


              Name      Suit  Points
0  Ace of Diamonds  Diamonds     0.0
1      2 of Spades    Spades     0.0
2   King of Hearts    Hearts     1.0
3      10 of Clubs     Clubs     1.1
4      8 of Hearts    Hearts     1.0


In [19]:
test_hand

NameError: name 'test_hand' is not defined

In [62]:
winner_index = test_hand[(test_hand['Suit'] == test_hand.iloc[0]['Suit']) & (test_hand['Points'] == test_hand['Points'].max())].index


In [63]:
winner_index

Index([], dtype='int64')

In [74]:
test_hand['Suit'].iloc[0]

'Diamonds'

In [75]:
current_hand

,Name,Suit,Points


In [18]:
play_card(list(test_hand['Name'][0]),player)

NameError: name 'test_hand' is not defined